# validator

> Fill in a module description here

In [ ]:
#| default_exp validator

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import re
from datetime import datetime
from typing import Any, Tuple, List, Union
from llm_data_extractor.models import Question, AnswerType
import logging

In [ ]:
#| export 

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) 

In [ ]:
#| export


def validate_answer(raw_answer: str, question: Question) -> Tuple[Any, bool, str]:
    """
    Validate and parse an answer according to question specifications.
    
    Args:
        raw_answer: Raw answer string from LLM
        question: Question object with validation rules
        
    Returns:
        Tuple of (parsed_answer, is_valid, error_message)
    """
    
    # Handle NULL responses
    if raw_answer.upper().strip() in ["NULL", "N/A", "NONE", ""]:
        return None, True, ""
    
    try:
        answer_type = question.answer_type
        answer_config = question.answer_config
        print(f'Answer Config: {answer_config}')
        
        if answer_type == AnswerType.BOOLEAN:
            return _validate_boolean(raw_answer, answer_config)
        elif answer_type == AnswerType.ENUM:
            return _validate_enum(raw_answer, answer_config)
        elif answer_type == AnswerType.INTEGER:
            return _validate_integer(raw_answer, answer_config)
        elif answer_type == AnswerType.FLOAT:
            return _validate_float(raw_answer, answer_config)
        elif answer_type == AnswerType.STRING:
            return _validate_string(raw_answer, answer_config)
        elif answer_type == AnswerType.DATE:
            return _validate_date(raw_answer, answer_config)
        elif answer_type == AnswerType.LIST:
            return _validate_list(raw_answer, answer_config)
        else:
            return raw_answer, False, f"Unknown answer type: {answer_type}"
            
    except Exception as e:
        return raw_answer, False, f"Validation error: {str(e)}"


def _validate_boolean(raw_answer: str, constraints: dict) -> Tuple[Union[bool,str], bool, str]:
    """Validate boolean answer."""
    answer = raw_answer.strip().lower()
    
    true_values = {"true", "yes", "1", "y", "t"}
    false_values = {"false", "no", "0", "n", "f"}
    
    if answer in true_values:
        return True, True, ""
    elif answer in false_values:
        return False, True, ""
    else:
        return raw_answer, False, f"Expected boolean value, got: {raw_answer}"


def _validate_enum(raw_answer: str, constraints: dict) -> Tuple[str, bool, str]:
    """Validate enum (categorical) answer."""
    valid_values = constraints.get("values", [])
    if not valid_values:
        return raw_answer, False, "No valid values specified for enum"
    
    # Convert all to strings for comparison
    valid_str = [str(v).strip() for v in valid_values]
    answer = raw_answer.strip()
    
    # Case-insensitive matching
    for valid_val in valid_str:
        if answer.lower() == valid_val.lower():
            return valid_val, True, ""
    
    return raw_answer, False, f"Value '{answer}' not in valid options: {valid_values}"


def _validate_integer(raw_answer: str, constraints: dict) -> Tuple[Union[bool,str,int], bool, str]:
    """Validate integer answer."""
    try:
        # Handle common formatting
        clean_answer = raw_answer.strip().replace(",", "")
        value = int(clean_answer)
        
        # Check bounds
        min_val = constraints.get("min")
        max_val = constraints.get("max")
        
        if min_val is not None and value < min_val:
            return value, False, f"Value {value} below minimum {min_val}"
        if max_val is not None and value > max_val:
            return value, False, f"Value {value} above maximum {max_val}"
            
        return value, True, ""
        
    except ValueError:
        return raw_answer, False, f"Cannot convert '{raw_answer}' to integer"


def _validate_float(raw_answer: str, constraints: dict) -> Tuple[Union[bool,str,float], bool, str]:
    """Validate float answer."""
    try:
        # Handle common formatting
        clean_answer = raw_answer.strip().replace(",", "")
        value = float(clean_answer)
        
        # Check bounds
        min_val = constraints.get("min")
        max_val = constraints.get("max")
        precision = constraints.get("precision")
        
        if min_val is not None and value < min_val:
            return value, False, f"Value {value} below minimum {min_val}"
        if max_val is not None and value > max_val:
            return value, False, f"Value {value} above maximum {max_val}"
        
        # Round to specified precision
        if precision is not None:
            value = round(value, precision)
            
        return value, True, ""
        
    except ValueError:
        return raw_answer, False, f"Cannot convert '{raw_answer}' to float"


def _validate_string(raw_answer: str, constraints: dict) -> Tuple[str, bool, str]:
    """Validate string answer."""
    value = raw_answer.strip()
    
    # Check length constraints
    min_length = constraints.get("min_length", 0)
    max_length = constraints.get("max_length")
    
    if len(value) < min_length:
        return value, False, f"String too short (min {min_length} chars)"
    if max_length is not None and len(value) > max_length:
        return value, False, f"String too long (max {max_length} chars)"
    
    # Check regex pattern if specified
    pattern = constraints.get("pattern")
    if pattern and not re.match(pattern, value):
        return value, False, f"String doesn't match required pattern"
    
    return value, True, ""


def _validate_date(raw_answer: str, constraints: dict) -> Tuple[Union[bool,str,datetime], bool, str]:
    """Validate date answer."""
    date_format = constraints.get("format", "%Y-%m-%d")
    
    try:
        value = datetime.strptime(raw_answer.strip(), date_format)
        
        # Check date range if specified
        min_date = constraints.get("min_date")
        max_date = constraints.get("max_date")
        
        if min_date and value < min_date:
            return value, False, f"Date before minimum {min_date}"
        if max_date and value > max_date:
            return value, False, f"Date after maximum {max_date}"
            
        return value, True, ""
        
    except ValueError:
        return raw_answer, False, f"Invalid date format. Expected {date_format}"


def _validate_list(raw_answer: str, constraints: dict) -> Tuple[List[Any], bool, str]:
    """Validate list answer."""
    separator = constraints.get("separator", ",")
    item_type = constraints.get("item_type", "string")
    min_items = constraints.get("min_items", 0)
    max_items = constraints.get("max_items")
    
    # Split the answer
    items = [item.strip() for item in raw_answer.split(separator) if item.strip()]
    
    # Check item count
    if len(items) < min_items:
        return items, False, f"Too few items (min {min_items})"
    if max_items is not None and len(items) > max_items:
        return items, False, f"Too many items (max {max_items})"
    
    # Validate each item based on type
    parsed_items = []
    for item in items:
        if item_type == "integer":
            try:
                parsed_items.append(int(item))
            except ValueError:
                return items, False, f"Invalid integer in list: {item}"
        elif item_type == "float":
            try:
                parsed_items.append(float(item))
            except ValueError:
                return items, False, f"Invalid float in list: {item}"
        else:  # string
            parsed_items.append(item)
    
    return parsed_items, True, ""

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()